In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from tqdm import tqdm, trange

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -q -U trl transformers git+https://github.com/huggingface/peft.git
!pip install bitsandbytes
!pip install accelerate

In [ ]:
import os
import torch
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer,\
BitsAndBytesConfig, AutoTokenizer
from datasets import Dataset
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model

In [ ]:
PATH = '/kaggle/input/combined/' 
summaries_train = pd.read_csv(f'{PATH}/Combined.csv')
summaries_train.head()

In [ ]:
summaries_train.shape

In [ ]:
summaries_train.nunique()

In [ ]:
import pandas as pd

PATH = '/kaggle/input/combined/Combined.csv'
df = pd.read_csv(PATH)

print(df.head())

def create_prompt_and_response(row):
    if row['anomaly'] == 'yes':
        prompt = f"When did device {row['Device_id']} show an anomaly?"
        response = f"The device {row['Device_id']} showed an anomaly on {row['timestamp']}."
    else:
        prompt = f"When did device {row['Device_id']} show an anomaly?"
        response = f"The device {row['Device_id']} did not show any anomaly recently."
    return prompt, response

df[['prompt', 'response']] = df.apply(create_prompt_and_response, axis=1, result_type='expand')

from datasets import Dataset
hf_dataset = Dataset.from_pandas(df[['prompt', 'response']])


In [ ]:
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

model.config.use_cache = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 50
logging_steps = 5
learning_rate = 2e-4
max_grad_norm = 0.3
num_train_epochs = 0.001 # Number of epochs
max_steps = None  # Steps, not epochs
warmup_ratio = 0.03
lr_scheduler_type = "constant"

max_seq_length = 1024

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    num_train_epochs=num_train_epochs,  # Set number of epochs
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)


In [ ]:
from datasets import load_metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    metric = load_metric("accuracy")
    return metric.compute(predictions=preds, references=labels)

trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    eval_dataset=hf_dataset,  # Add an evaluation dataset
    peft_config=peft_config,
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    compute_metrics=compute_metrics  # Include the compute_metrics function
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("final_finetuned_model")

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="final_finetuned_model", tokenizer=tokenizer)

prompt = "Does  device f66f1a4a-e7db-41d4-aec8-f7392ce5ab11 show an anomaly on 2023-01-11 "
response = pipe(prompt, max_length=50)
print(response[0]['generated_text'])
